# D210 - Reporting and Representation

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell 

pd.set_option('display.max_columns', None)
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

In [2]:
df = pd.read_csv('medical_clean.csv', index_col='Customer_id')

In [3]:
df.shape

(10000, 49)

In [4]:
df.head()

,CaseOrder,Interaction,UID,City,State,County,Zip,Lat,Lng,Population,Area,TimeZone,Job,Children,Age,Income,Marital,Gender,ReAdmis,VitD_levels,Doc_visits,Full_meals_eaten,vitD_supp,Soft_drink,Initial_admin,HighBlood,Stroke,Complication_risk,Overweight,Arthritis,Diabetes,Hyperlipidemia,BackPain,Anxiety,Allergic_rhinitis,Reflux_esophagitis,Asthma,Services,Initial_days,TotalCharge,Additional_charges,Item1,Item2,Item3,Item4,Item5,Item6,Item7,Item8
Customer_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C412403,1,8cd49b13-f45a-4b47-a2bd-173ffa932c2f,3a83ddb66e2ae73798bdf1d705dc0932,Eva,AL,Morgan,35621,34.34960,-86.72508,2951,Suburban,America/Chicago,"Psychologist, sport and exercise",1,53,86575.93,Divorced,Male,No,19.141466,6,0,0,No,Emergency Admission,Yes,No,Medium,No,Yes,Yes,No,Yes,Yes,Yes,No,Yes,Blood Work,10.585770,3726.702860,17939.403420,3,3,2,2,4,3,3,4
Z919181,2,d2450b70-0337-4406-bdbb-bc1037f1734c,176354c5eef714957d486009feabf195,Marianna,FL,Jackson,32446,30.84513,-85.22907,11303,Urban,America/Chicago,Community development worker,3,51,46805.99,Married,Female,No,18.940352,4,2,1,No,Emergency Admission,Yes,No,High,Yes,No,No,No,No,No,No,Yes,No,Intravenous,15.129562,4193.190458,17612.998120,3,4,3,4,4,4,3,3
F995323,3,a2057123-abf5-4a2c-abad-8ffe33512562,e19a0fa00aeda885b8a436757e889bc9,Sioux Falls,SD,Minnehaha,57110,43.54321,-96.63772,17125,Suburban,America/Chicago,Chief Executive Officer,3,53,14370.14,Widowed,Female,No,18.057507,4,1,0,No,Elective Admission,Yes,No,Medium,Yes,No,Yes,No,No,No,No,No,No,Blood Work,4.772177,2434.234222,17505.192460,2,4,4,4,3,4,3,3
A879973,4,1dec528d-eb34-4079-adce-0d7a40e82205,cd17d7b6d152cb6f23957346d11c3f07,New Richland,MN,Waseca,56072,43.89744,-93.51479,2162,Suburban,America/Chicago,Early years teacher,0,78,39741.49,Married,Male,No,16.576858,4,1,0,No,Elective Admission,No,Yes,Medium,No,Yes,No,No,No,No,No,Yes,Yes,Blood Work,1.714879,2127.830423,12993.437350,3,5,5,3,4,5,5,5
C544523,5,5885f56b-d6da-43a3-8760-83583af94266,d2f0425877b10ed6bb381f3e2579424a,West Point,VA,King William,23181,37.59894,-76.88958,5287,Rural,America/New_York,Health promotion specialist,1,22,1209.56,Widowed,Female,No,17.439069,5,0,2,Yes,Elective Admission,No,No,Low,No,No,No,Yes,No,No,Yes,No,No,CT Scan,1.254807,2113.073274,3716.525786,2,1,3,3,5,3,4,3


In [ ]:
# df.drop(['CaseOrder', 'Interaction', 'UID', 'Lat', 'Lng'], axis=1, inplace=True)

In [ ]:
# df.head()

In [5]:
survey_df = df[['Item1', 'Item2', 'Item3', 'Item4', 'Item5', 'Item6', 'Item7', 'Item8']]

In [6]:
survey_df.rename(columns={'Item1':'Timely_Admission', 'Item2':'Timely_Treatment', 'Item3':'Timely_Visits', 'Item4':'Reliability',
                          'Item5':'Options', 'Item6':'HoursofTreatment', 'Item7':'Courteous_Staff', 'Item8':'Active_Listening_Doctor'}, inplace=True)

C:\Users\Bader Ale\AppData\Local\Temp\ipykernel_18424\2575861863.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  survey_df.rename(columns={'Item1':'Timely_Admission', 'Item2':'Timely_Treatment', 'Item3':'Timely_Visits', 'Item4':'Reliability',


In [11]:
df.drop(['Item1','Item2','Item3','Item4','Item5','Item6','Item7','Item8'], axis=1, inplace=True)

In [12]:
survey_df.head()

,Timely_Admission,Timely_Treatment,Timely_Visits,Reliability,Options,HoursofTreatment,Courteous_Staff,Active_Listening_Doctor
Customer_id,,,,,,,,
C412403,3,3,2,2,4,3,3,4
Z919181,3,4,3,4,4,4,3,3
F995323,2,4,4,4,3,4,3,3
A879973,3,5,5,3,4,5,5,5
C544523,2,1,3,3,5,3,4,3


# § Data Cleaning  

### ▶ Detection and Treatment of Null Values

In [13]:
df.isnull().sum()

CaseOrder             0
Interaction           0
UID                   0
City                  0
State                 0
County                0
Zip                   0
Lat                   0
Lng                   0
Population            0
Area                  0
TimeZone              0
Job                   0
Children              0
Age                   0
Income                0
Marital               0
Gender                0
ReAdmis               0
VitD_levels           0
Doc_visits            0
Full_meals_eaten      0
vitD_supp             0
Soft_drink            0
Initial_admin         0
HighBlood             0
Stroke                0
Complication_risk     0
Overweight            0
Arthritis             0
Diabetes              0
Hyperlipidemia        0
BackPain              0
Anxiety               0
Allergic_rhinitis     0
Reflux_esophagitis    0
Asthma                0
Services              0
Initial_days          0
TotalCharge           0
Additional_charges    0
dtype: int64

### ▶ Detection and Treatment of Duplicated Values

In [14]:
df.duplicated().value_counts()

False    10000
Name: count, dtype: int64

# § Readmission Prediction using Random Forest

In [18]:
# Creating X and y data
X = df[['Area','Income','Marital','Gender','VitD_levels','Doc_visits',
       'Full_meals_eaten','vitD_supp','Soft_drink','Initial_admin',
       'HighBlood','Stroke','Complication_risk','Overweight','Arthritis',
       'Diabetes','Hyperlipidemia','BackPain','Anxiety','Allergic_rhinitis',
       'Reflux_esophagitis','Asthma','Services','Initial_days','TotalCharge',
        'Additional_charges']]
y = df['ReAdmis'].values.reshape(-1,1)

In [19]:
X.head()

,Area,Income,Marital,Gender,VitD_levels,Doc_visits,Full_meals_eaten,vitD_supp,Soft_drink,Initial_admin,HighBlood,Stroke,Complication_risk,Overweight,Arthritis,Diabetes,Hyperlipidemia,BackPain,Anxiety,Allergic_rhinitis,Reflux_esophagitis,Asthma,Services,Initial_days,TotalCharge,Additional_charges
Customer_id,,,,,,,,,,,,,,,,,,,,,,,,,,
C412403,Suburban,86575.93,Divorced,Male,19.141466,6,0,0,No,Emergency Admission,Yes,No,Medium,No,Yes,Yes,No,Yes,Yes,Yes,No,Yes,Blood Work,10.585770,3726.702860,17939.403420
Z919181,Urban,46805.99,Married,Female,18.940352,4,2,1,No,Emergency Admission,Yes,No,High,Yes,No,No,No,No,No,No,Yes,No,Intravenous,15.129562,4193.190458,17612.998120
F995323,Suburban,14370.14,Widowed,Female,18.057507,4,1,0,No,Elective Admission,Yes,No,Medium,Yes,No,Yes,No,No,No,No,No,No,Blood Work,4.772177,2434.234222,17505.192460
A879973,Suburban,39741.49,Married,Male,16.576858,4,1,0,No,Elective Admission,No,Yes,Medium,No,Yes,No,No,No,No,No,Yes,Yes,Blood Work,1.714879,2127.830423,12993.437350
C544523,Rural,1209.56,Widowed,Female,17.439069,5,0,2,Yes,Elective Admission,No,No,Low,No,No,No,Yes,No,No,Yes,No,No,CT Scan,1.254807,2113.073274,3716.525786


In [20]:
X = pd.get_dummies(data=X, columns=['Area','Marital','Gender','Soft_drink',
                                    'Initial_admin','HighBlood','Stroke','Overweight','Arthritis',
                                    'Diabetes','Hyperlipidemia','BackPain','Anxiety','Allergic_rhinitis',
                                    'Reflux_esophagitis','Asthma','Services'], drop_first=True)


In [22]:
X.head()

,Income,VitD_levels,Doc_visits,Full_meals_eaten,vitD_supp,Complication_risk,Initial_days,TotalCharge,Additional_charges,Area_Suburban,Area_Urban,Marital_Married,Marital_Never Married,Marital_Separated,Marital_Widowed,Gender_Male,Gender_Nonbinary,Soft_drink_Yes,Initial_admin_Emergency Admission,Initial_admin_Observation Admission,HighBlood_Yes,Stroke_Yes,Overweight_Yes,Arthritis_Yes,Diabetes_Yes,Hyperlipidemia_Yes,BackPain_Yes,Anxiety_Yes,Allergic_rhinitis_Yes,Reflux_esophagitis_Yes,Asthma_Yes,Services_CT Scan,Services_Intravenous,Services_MRI
Customer_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C412403,86575.93,19.141466,6,0,0,Medium,10.585770,3726.702860,17939.403420,True,False,False,False,False,False,True,False,False,True,False,True,False,False,True,True,False,True,True,True,False,True,False,False,False
Z919181,46805.99,18.940352,4,2,1,High,15.129562,4193.190458,17612.998120,False,True,True,False,False,False,False,False,False,True,False,True,False,True,False,False,False,False,False,False,True,False,False,True,False
F995323,14370.14,18.057507,4,1,0,Medium,4.772177,2434.234222,17505.192460,True,False,False,False,False,True,False,False,False,False,False,True,False,True,False,True,False,False,False,False,False,False,False,False,False
A879973,39741.49,16.576858,4,1,0,Medium,1.714879,2127.830423,12993.437350,True,False,True,False,False,False,True,False,False,False,False,False,True,False,True,False,False,False,False,False,True,True,False,False,False
C544523,1209.56,17.439069,5,0,2,Low,1.254807,2113.073274,3716.525786,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,True,False,False,True,False,False,True,False,False


In [27]:
# Encoding colum "Complication Risk"
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
X['Complication_risk'] = enc.fit_transform(X[['Complication_risk']])

In [28]:
X.head()

,Income,VitD_levels,Doc_visits,Full_meals_eaten,vitD_supp,Complication_risk,Initial_days,TotalCharge,Additional_charges,Area_Suburban,Area_Urban,Marital_Married,Marital_Never Married,Marital_Separated,Marital_Widowed,Gender_Male,Gender_Nonbinary,Soft_drink_Yes,Initial_admin_Emergency Admission,Initial_admin_Observation Admission,HighBlood_Yes,Stroke_Yes,Overweight_Yes,Arthritis_Yes,Diabetes_Yes,Hyperlipidemia_Yes,BackPain_Yes,Anxiety_Yes,Allergic_rhinitis_Yes,Reflux_esophagitis_Yes,Asthma_Yes,Services_CT Scan,Services_Intravenous,Services_MRI
Customer_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C412403,86575.93,19.141466,6,0,0,2.0,10.585770,3726.702860,17939.403420,True,False,False,False,False,False,True,False,False,True,False,True,False,False,True,True,False,True,True,True,False,True,False,False,False
Z919181,46805.99,18.940352,4,2,1,0.0,15.129562,4193.190458,17612.998120,False,True,True,False,False,False,False,False,False,True,False,True,False,True,False,False,False,False,False,False,True,False,False,True,False
F995323,14370.14,18.057507,4,1,0,2.0,4.772177,2434.234222,17505.192460,True,False,False,False,False,True,False,False,False,False,False,True,False,True,False,True,False,False,False,False,False,False,False,False,False
A879973,39741.49,16.576858,4,1,0,2.0,1.714879,2127.830423,12993.437350,True,False,True,False,False,False,True,False,False,False,False,False,True,False,True,False,False,False,False,False,True,True,False,False,False
C544523,1209.56,17.439069,5,0,2,1.0,1.254807,2113.073274,3716.525786,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,True,False,False,True,False,False,True,False,False


In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=53)

# rfc = RandomForestClassifier(random_state=53)
# rfc.fit(X_train,y_train)


In [33]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=53)

pipe = Pipeline([
        ('scaler', RobustScaler()),
        ('model', RandomForestClassifier())])

pipe.get_params()

{'memory': None,
 'steps': [('scaler', RobustScaler()), ('model', RandomForestClassifier())],
 'verbose': False,
 'scaler': RobustScaler(),
 'model': RandomForestClassifier(),
 'scaler__copy': True,
 'scaler__quantile_range': (25.0, 75.0),
 'scaler__unit_variance': False,
 'scaler__with_centering': True,
 'scaler__with_scaling': True,
 'model__bootstrap': True,
 'model__ccp_alpha': 0.0,
 'model__class_weight': None,
 'model__criterion': 'gini',
 'model__max_depth': None,
 'model__max_features': 'sqrt',
 'model__max_leaf_nodes': None,
 'model__max_samples': None,
 'model__min_impurity_decrease': 0.0,
 'model__min_samples_leaf': 1,
 'model__min_samples_split': 2,
 'model__min_weight_fraction_leaf': 0.0,
 'model__n_estimators': 100,
 'model__n_jobs': None,
 'model__oob_score': False,
 'model__random_state': None,
 'model__verbose': 0,
 'model__warm_start': False}

In [36]:
mod = GridSearchCV(estimator=pipe,
                   param_grid={'model__max_depth': [1,2,3,4,5,6,7,8,9,10]},
                   cv=5,
                   n_jobs=-1)

In [37]:
mod.fit(X_train, y_train)

c:\Users\Bader Ale\anaconda3\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', RobustScaler()),
                                       ('model', RandomForestClassifier())]),
             n_jobs=-1,
             param_grid={'model__max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]})

In [39]:
pd.DataFrame(mod.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.657725,0.065268,0.059550,0.016982,1,{'model__max_depth': 1},0.960000,0.630714,0.971429,0.964286,0.952143,0.895714,0.132647,10
1,0.840921,0.071065,0.050121,0.008507,2,{'model__max_depth': 2},0.973571,0.971429,0.972143,0.975714,0.970000,0.972571,0.001948,9
2,1.096170,0.140969,0.061129,0.019555,3,{'model__max_depth': 3},0.982143,0.980714,0.977857,0.977857,0.970714,0.977857,0.003938,8
3,1.262337,0.127532,0.057687,0.007311,4,{'model__max_depth': 4},0.984286,0.982143,0.978571,0.980714,0.972857,0.979714,0.003902,7
4,1.469086,0.210944,0.059726,0.008508,5,{'model__max_depth': 5},0.985000,0.985000,0.979286,0.980000,0.972857,0.980429,0.004486,5
5,1.559975,0.170236,0.062207,0.012605,6,{'model__max_depth': 6},0.984286,0.985000,0.980000,0.981429,0.971429,0.980429,0.004857,6
6,1.785558,0.197290,0.066223,0.010043,7,{'model__max_depth': 7},0.983571,0.987143,0.980000,0.980714,0.972143,0.980714,0.004969,1
7,1.738871,0.218002,0.060060,0.008203,8,{'model__max_depth': 8},0.982857,0.985714,0.982143,0.981429,0.970714,0.980571,0.005139,3
8,1.834948,0.086765,0.053195,0.005870,9,{'model__max_depth': 9},0.983571,0.985714,0.980000,0.981429,0.972857,0.980714,0.004380,1
9,1.664656,0.119306,0.034445,0.005225,10,{'model__max_depth': 10},0.982857,0.987143,0.980000,0.979286,0.973571,0.980571,0.004458,3


In [47]:
print(f'The best depth for the Random Forest is: {mod.best_params_}')
print(f'The best score was : {mod.best_score_}')


The best depth for the Random Forest is: {'model__max_depth': 7}
The best score was : 0.9807142857142856
